## Data Preparation

In [231]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from pymongo import MongoClient
import json, re
from re import sub
from decimal import Decimal
import dask.dataframe as dd
from dask.distributed import Client, progress
import dask.dataframe as dd
from dask.delayed import delayed

### Dask Client

In [265]:
client = Client(n_workers=4, threads_per_worker=1)
client

Client Scheduler: tcp://127.0.0.1:45551 Dashboard: http://127.0.0.1:40293/status,Cluster Workers: 4 Cores: 4 Memory: 8.26 GB


### Database Connection

In [233]:
DB_USER = 'anutibara'
DB_PASS = 'anutibara'
DB_HOST = 'scraping-cluster-7dtgt.gcp.mongodb.net'
DB_NAME = 'tes_scraping_db'

try:
    client = MongoClient(f'mongodb+srv://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}')
    print("Database connected successfully")
except Exception as e:
    print("Error to connect to database: ", e)
db = client.get_database(DB_NAME)
properties = db.properties

Database connected successfully


### Database Queries

In [234]:
properties.count_documents({})

67

### Convert Mongo Collection to DataFrame

#### New Properties

In [235]:
new_properties_json = list(properties.find({ 'use': 'Nuevo' }))

In [236]:
df_general_info = json_normalize(new_properties_json, record_path='offersType', 
                                meta=['_id', 
                                    'urlProperty',
                                    'scrapingDate', 
                                    'scrapingHour', 
                                    'modifyDate',
                                    'modifyHour', 
                                    'code', 
                                    'status', 
                                    'type', 
                                    'use', 
                                    'nameProject', 
                                    'description'])
ddf_general_info = dd.from_pandas(df_general_info, npartitions=10)

In [237]:
df_location = json_normalize(new_properties_json, record_path='location', meta='urlProperty')
ddf_location = dd.from_pandas(df_location, npartitions=10)

In [238]:
ddf_1 = dd.merge(ddf_general_info, ddf_location, on='urlProperty')

In [239]:
df_owner_property = json_normalize(new_properties_json, record_path='ownerProperty', meta='urlProperty')
ddf_owner_property = dd.from_pandas(df_owner_property, npartitions=10)

In [240]:
ddf_2 = dd.merge(ddf_1, ddf_owner_property, on='urlProperty')

In [241]:
df_features = json_normalize(new_properties_json, record_path='features', meta='urlProperty')
ddf_features = dd.from_pandas(df_features, npartitions=10)

In [242]:
ddf_features = ddf_features.rename(columns={
    'price':'range_prices',
    'rooms': 'general_rooms',
    'bathrooms': 'general_bathrooms',
    'privateArea': 'range_private_area'
})

In [243]:
ddf_3 = dd.merge(ddf_2, ddf_features, on='urlProperty')

In [244]:
df_more_features = json_normalize(new_properties_json, record_path='moreFeatures', meta='urlProperty')
ddf_more_features = dd.from_pandas(df_more_features, npartitions=10)

In [245]:
ddf_4 = dd.merge(ddf_3, ddf_more_features, on='urlProperty')

In [246]:
ddf_new_properties = ddf_4.loc[:,~ddf_4.columns.duplicated(keep='first')]

In [247]:
ddf_new_properties = ddf_new_properties.dropna()

#### Old Properties

In [248]:
old_properties_json = list(properties.find({ 'use': 'Usado' }))

In [249]:
df_general_info = pd.DataFrame(old_properties_json, 
                                    columns=['_id', 
                                    'urlProperty', 
                                    'scrapingDate', 
                                    'scrapingHour', 
                                    'modifyDate',
                                    'modifyHour', 
                                    'code', 
                                    'status', 
                                    'type', 
                                    'use', 
                                    'nameProject', 
                                    'description'])
ddf_general_info = dd.from_pandas(df_general_info, npartitions=10)

In [250]:
df_location = json_normalize(old_properties_json, 'location')
ddf_location = dd.from_pandas(df_location, npartitions=10)

In [251]:
df_owner_property = json_normalize(old_properties_json, 'ownerProperty')
ddf_owner_property = dd.from_pandas(df_owner_property, npartitions=10)

In [252]:
df_features = json_normalize(old_properties_json, 'features')
ddf_features = dd.from_pandas(df_features, npartitions=10)

In [253]:
df_more_features = json_normalize(old_properties_json, 'moreFeatures')
ddf_more_features = dd.from_pandas(df_more_features, npartitions=10)

In [254]:
ddf_list = [ddf_general_info, ddf_location, ddf_owner_property, ddf_features, ddf_more_features]
ddf = dd.concat(ddf_list, axis=1)
ddf.reset_index()

,index,_id,urlProperty,scrapingDate,scrapingHour,modifyDate,modifyHour,code,status,type,use,nameProject,description,address,city,country,department,latitude,longitude,neighborhood,sector,contractType,financing,id,name,schedule,admonPrice,antiquity,bathrooms,condition,constructionArea,floor,garages,includesAdministration,interiorFloors,price,privateArea,rooms,squareMeters,squareMetersPrice,stratum,weather,exteriorFeatures,interiorFeatures,sectorFeatures
npartitions=16,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,object,object,object,object,int64,object,object,object,object,object,object,object,object,object,float64,float64,object,object,object,object,int64,object,object,float64,object,int64,object,object,object,object,object,int64,object,object,int64,object,float64,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [255]:
ddf_old_properties = ddf.loc[:,~ddf.columns.duplicated()]

In [256]:
ddf_old_properties = ddf_old_properties.dropna()

### Rename DataFrame Columns

#### New Properties

In [257]:
ddf_new_properties = ddf_new_properties.rename(columns={
                        "_id": "id_mongoose", 
                        "urlProperty":"id_property",
                        "scrapingDate": "scraping_date",
                        "scrapingHour": "scraping_hour",
                        "modifyDate": "modify_date",
                        "modifyHour": "modify_hour",
                        "status": "active",
                        "use": "new_property",
                        "nameProject": "name_project",
                        "offersType": "offers_type",
                        "id": "id_owner_property",
                        "name": "name_owner_property",
                        "contractType": "contract_type_owner_property",
                        "financing": "financing_owner_property",
                        "schedule": "schedule_owner_property",
                        "squareMeters": "range_square_meters",
                        "constructionArea": "range_construction_area",
                        "squareMetersPrice": "square_meters_price",
                        "interiorFloors": "interior_floors",
                        "includesAdministration": "includes_administration",
                        "admonPrice": "admon_price",
                        "interiorFeatures": "interior_features",
                        "exteriorFeatures": "exterior_features",
                        "sectorFeatures": "sector_features",
                        "offerType": "offer_type",
                        "privateArea": "private_area"
})

#### Old Properties

In [258]:
ddf_old_properties = ddf_old_properties.rename(columns={
                        "_id": "id_mongoose", 
                        "urlProperty":"id_property",
                        "scrapingDate": "scraping_date",
                        "scrapingHour": "scraping_hour",
                        "modifyDate": "modify_date",
                        "modifyHour": "modify_hour",
                        "status": "active",
                        "use": "new_property",
                        "nameProject": "name_project",
                        "offersType": "offers_type",
                        "id": "id_owner_property",
                        "name": "name_owner_property",
                        "contractType": "contract_type_owner_property",
                        "financing": "financing_owner_property",
                        "schedule": "schedule_owner_property",
                        "squareMeters": "square_meters",
                        "privateArea": "private_area",
                        "constructionArea": "construction_area",
                        "squareMetersPrice": "square_meters_price",
                        "interiorFloors": "interior_floors",
                        "includesAdministration": "includes_administration",
                        "admonPrice": "admon_price",
                        "interiorFeatures": "interior_features",
                        "exteriorFeatures": "exterior_features",
                        "sectorFeatures": "sector_features"
})

### Data Cleaning

#### New Properties

In [259]:
ddf_new_properties['id_mongoose'] = ddf_new_properties['id_mongoose'].astype(str)
ddf_new_properties['code'] = ddf_new_properties['code'].astype(int)
ddf_new_properties['active'] = ddf_new_properties['active'].apply(lambda status: True if (status == 'Active') else False, meta=('active', 'bool'))
ddf_new_properties['new_property'] = ddf_new_properties['new_property'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('new_property', 'bool'))
ddf_new_properties['includes_administration'] = ddf_new_properties['includes_administration'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('includes_administration', 'bool'))
ddf_new_properties['garages'] = ddf_new_properties['garages'].replace('', value = 0, regex = True).astype(int)
ddf_new_properties['stratum'] = ddf_new_properties['stratum'].replace('', value = 0, regex = True).astype(int)
ddf_new_properties['floor'] = ddf_new_properties['floor'].astype(int)
ddf_new_properties['area'] = ddf_new_properties['area'].astype(float)
ddf_new_properties['private_area'] = ddf_new_properties['private_area'].replace('', value = 0, regex = True)
ddf_new_properties['private_area'] = ddf_new_properties['private_area'].astype(float)
ddf_new_properties['rooms'] = ddf_new_properties['rooms'].astype(int)
ddf_new_properties['bathrooms'] = ddf_new_properties['bathrooms'].astype(int)
ddf_new_properties['price'] = ddf_new_properties['price'].apply(lambda price: Decimal(sub(r'[^\d,]', '', price)), meta=('price', 'int'))
ddf_new_properties['price'] = ddf_new_properties['price'].astype(int)

#### Old Properties

In [260]:
ddf_old_properties['id_mongoose'] = ddf_old_properties['id_mongoose'].astype(str)
ddf_old_properties['code'] = ddf_old_properties['code'].astype(int)
ddf_old_properties['active'] = ddf_old_properties['active'].apply(lambda status: True if (status == 'Active') else False, meta=('active', 'bool'))
ddf_old_properties['new_property'] = ddf_old_properties['new_property'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('new_property', 'bool'))
ddf_old_properties['includes_administration'] = ddf_old_properties['includes_administration'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('includes_administration', 'bool'))
ddf_old_properties['garages'] = ddf_old_properties['garages'].replace('', value = 0, regex = True)
ddf_old_properties['garages'] = ddf_old_properties['garages'].replace('Más de 10', value = 0, regex = True)
ddf_old_properties['garages'] = ddf_old_properties['garages'].astype(int)
ddf_old_properties['stratum'] = ddf_old_properties['stratum'].replace('', value = 0, regex = True)
ddf_old_properties['stratum'] = ddf_old_properties['stratum'].replace('Campestre', value = 0, regex = True)
ddf_old_properties['stratum'] = ddf_old_properties['stratum'].astype(int)
ddf_old_properties['price'] = ddf_old_properties['price'].apply(lambda price: Decimal(sub(r'[^\d,]', '', price)), meta=('price', 'int'))
ddf_old_properties['price'] = ddf_old_properties['price'].astype(int)
ddf_old_properties['square_meters'] = ddf_old_properties['square_meters'].str[0:-3]
ddf_old_properties['square_meters'] = ddf_old_properties['square_meters'].apply(lambda meters: Decimal(sub(r'[^\d,]', '', meters)), meta=('square_meters', 'float'))
ddf_old_properties['square_meters'] = ddf_old_properties['square_meters'].astype(float)
ddf_old_properties['private_area'] = ddf_old_properties['private_area'].str[0:-2]
ddf_old_properties['private_area'] = ddf_old_properties['private_area'].replace('', value = '0', regex = True)
ddf_old_properties['private_area'] = ddf_old_properties['private_area'].apply(lambda area: Decimal(sub(r'[^\d.]', '', area)), meta=('private_area', 'float'))
ddf_old_properties['private_area'] = ddf_old_properties['private_area'].astype(float)
ddf_old_properties['construction_area'] = ddf_old_properties['construction_area'].str[0:-3]
ddf_old_properties['construction_area'] = ddf_old_properties['construction_area'].apply(lambda area: Decimal(sub(r'[^\d,]', '', area)), meta=('construction_area', 'float'))
ddf_old_properties['construction_area'] = ddf_old_properties['construction_area'].astype(float)
ddf_old_properties['floor'] = ddf_old_properties['floor'].astype(int)

### Testing

#### New Properties

In [261]:
ddf_new_properties.describe().head()

,area,bathrooms,price,private_area,rooms,code,latitude,longitude,id_owner_property,admon_price,general_bathrooms,floor,garages,interior_floors,general_rooms,square_meters_price,stratum
count,96.000000,96.000000,9.600000e+01,96.000000,96.000000,9.600000e+01,96.000000,96.000000,96.000000,96.0,96.000000,96.0,96.000000,96.0,96.000000,9.600000e+01,96.000000
mean,85.985104,2.864583,3.919729e+08,76.055208,2.989583,2.835672e+06,5.840435,-70.842508,391989.583333,0.0,2.843750,0.0,0.864583,0.0,2.916667,4.488742e+06,4.020833
std,5.881137,0.343964,3.969747e+07,20.455519,0.229177,2.022698e+05,1.515955,18.387480,68670.723618,0.0,0.364998,0.0,0.343964,0.0,0.401751,1.779278e+05,0.143576
min,58.790000,2.000000,2.507270e+08,0.000000,1.000000,2.393445e+06,0.000000,-75.602592,350013.000000,0.0,2.000000,0.0,0.000000,0.0,1.000000,4.043984e+06,4.000000
25%,87.500000,3.000000,3.966325e+08,82.900000,3.000000,2.800362e+06,6.225793,-75.561203,383498.000000,0.0,3.000000,0.0,1.000000,0.0,3.000000,4.521514e+06,4.000000


#### Old Properties

In [262]:
ddf_old_properties.describe().head()

,code,latitude,longitude,id_owner_property,admon_price,bathrooms,construction_area,floor,garages,interior_floors,price,private_area,rooms,square_meters,square_meters_price,stratum
count,4.000000e+01,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,4.000000e+01,40.000000,40.000000,40.000000,4.000000e+01,40.000000
mean,4.941382e+06,5.714063,-69.876966,594606.225000,122312.375000,2.475000,154.275000,2.500000,1.100000,0.075000,3.176608e+08,62.162500,2.375000,154.275000,2.312436e+06,4.525000
std,1.956021e+05,1.666792,20.152153,565894.251534,213784.447271,1.085747,230.163151,4.607686,1.780233,0.349908,4.913495e+08,74.970797,1.462129,230.163151,2.338529e+06,1.601081
min,4.310836e+06,0.000000,-75.630013,61564.000000,0.000000,1.000000,32.000000,0.000000,0.000000,0.000000,1.300000e+06,0.000000,0.000000,32.000000,2.333330e+03,1.000000
25%,4.953315e+06,6.201599,-75.587355,329103.500000,0.000000,2.000000,86.750000,0.000000,0.000000,0.000000,3.100000e+06,13.250000,2.000000,86.750000,4.467366e+04,4.250000
